# 1. Box Cox and Derivation
# 2. Test Dickey-Fuller
# 3. Autocorrelation plot
# 4.
# 5.
# 6.
# 7.
# 8.
# 9.
# 10.
# 11.
# 12.
# 13.
# 14.
# 15.

# 1. Box Cox and Derivation (for getting a stationary time series)

In [ ]:
# 1. Box Cox Преобразование Бокса-Кокса часто применяется для нормализации данных или для стабилизации дисперсии
from scipy.stats import boxcox  
from scipy.special import inv_boxcox # для обратного преобразования

all_series = {
    "Monthly sales of company X": sales_of_company_x["Count"],
    "International airline passengers: monthly totals in thousands": airlines_passengers["Count"]
}

# boxcox(x, lmbda=None, alpha=None)  lmbda (необязательный)  - степень трансформации;  alpha: (необязательный) уровень значимости для доверительного интервала lmbda. 
series = boxcox(
    x=all_series["International airline passengers: monthly totals in thousands"], 
    lmbda=0,
    alpha=None
)


inv = inv_boxcox(series, lmbda=0)

In [ ]:
# 2. Derivation
import numpy as np 

# np.diff для вычисления разностей между элементами массива вдоль указанной оси. 
# для анализа временных рядов, вычисления производных или обнаружения изменений между последовательными значениями.
# np.diff(a, n=1, axis=-1) n: (по умолчанию 1) количество разностей, которые нужно вычислить. Например, n=2 вернет разности второго порядка.
# axis: (по умолчанию -1) ось, вдоль которой вычисляются разности.
series = np.diff(series, 1)


# or 
series = series[1:] - series[:-1]
# or
series = series[12:] - series[:-12]



# 2. Test Dickey-Fuller (to test if the time series is stationary)

In [ ]:
from statsmodels.tsa.stattools import adfuller

adfuller(x, maxlag=None, regression='c', autolag='AIC', store=False, regresults=False)
# x: список, массив NumPy или серия pandas с временным рядом.
# maxlag: максимальное количество лагов
# regression: тип регрессии: тут 'c' для включения константы но есть и много других напр., 'ctt' для включения константы, линейного и квадратичного трендов,
# autolag: метод автоматического выбора лагов, тут 'AIC' для критерия Акаике
# store: если True, возвращает дополнительные результаты в виде класса с атрибутами.
# regresults: если True, сохраняет результаты промежуточной регрессии.


dftest = adfuller(timeseries, autolag='AIC')


# example of method
################
 # Dickey-Fuller
##################
def test_stationarity(timeseries):
    print('Results of Dickey-Fuller Test:')
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic', 'p-value', '#Lags Used', 'Number of Observations Used'])
    for [key, value] in dftest[4].items():
        dfoutput['Critical Value (%s)' % key] = value
    print(dfoutput)

# call method
test_stationarity(all_series["International airline passengers: monthly totals in thousands"])

In [ ]:
# !! HELPER METHOD for defining the seasonality SEASONALITY
def plot_ts_and_points(ts, start_point, step):
    new_series = [None for i in range(len(ts))]
    for i in range(len(ts)):
        pos = start_point + step * i
        if pos >= len(ts):
            break
        new_series[pos] = ts[pos]
    new_series = pd.Series(new_series)
    
    with plt.style.context('bmh'):
        plt.figure(figsize=(16, 8))
        ts_ax = plt.axes()
        ts.plot(ax=ts_ax, color='blue')
        new_series.plot(ax=ts_ax, style='ro')

# 3. Autocorrelation plot

In [ ]:
import statsmodels.api as sm
import statsmodels.tsa.api as smt

# statsmodels.api предоставляет высокоуровневый доступ ко всем функциям библиотеки statsmodels. Обычно используется для общей статистики и моделирования.
# statsmodels.tsa.api предоставляет доступ к функциям и классам, специфичным для анализа временных рядов. 
# Это включает в себя модели ARIMA, тесты на стационарность, функции для построения автокорреляционных графиков и многое другое.

# plot_acf - Автокорреляционная функция - идентифицирует зависимость значений временного ряда на разных лагах.
smt.graphics.plot_acf(y, lags=lags, ax=acf_ax, alpha=0.05)

# plot_pacf - Частичная автокорреляционная функция - показывает корреляцию значений временного ряда с лагами, устранение влияния промежуточных лагах.
smt.graphics.plot_pacf(y, lags=lags, ax=pacf_ax, alpha=0.05)


#  метод для построения коррелорграммы 
def tsplot(y, lags=None, figsize=(14, 8), style='bmh'):
    if not isinstance(y, pd.Series):
        y = pd.Series(y)
    with plt.style.context(style):
        plt.figure(figsize=figsize)
        layout = (4, 1)
        ts_ax = plt.subplot2grid(layout, (0, 0), rowspan=2)
        acf_ax = plt.subplot2grid(layout, (2, 0))
        pacf_ax = plt.subplot2grid(layout, (3, 0))

        y.plot(ax=ts_ax, color='blue', label='Or')
        ts_ax.set_title('Original')

        smt.graphics.plot_acf(y, lags=lags, ax=acf_ax, alpha=0.05)
        smt.graphics.plot_pacf(y, lags=lags, ax=pacf_ax, alpha=0.05)

        plt.tight_layout()
    return


tsplot(all_series["International airline passengers: monthly totals in thousands"])